In [7]:
import pandas as pd
import mysql.connector
import numpy as np
#import altair as alt
#from altair_saver import save # to save the altair graphs
#import selenium # to save the altair graphs
import os
import dropbox
#import knmi_gn_001 # get daily data from knmi
from datetime import date
from datetime import datetime, timedelta

In [8]:
def readsettings():
    # Read generic settings
    path = os.getcwd()
    path = "/home/pi/settings_GN"
    TOKENS = path+'/GN_values.txt'
    token_file = open(TOKENS)
    gn_user,gn_pw,gn_host,gn_dbase,gn_drpbx = token_file.read().split('|')
    token_file.close()
    a = {'user': gn_user,
         'pw': gn_pw,
         'host': gn_host,
         'dbase': gn_dbase,
         'drpbox': gn_drpbx}
#     return(gn_user,gn_pw,gn_host,gn_dbase,gn_drpbx)
    return(a)


In [9]:
class MinuteData:
    
    def __init__(self,name_str,dbase_str,fulldbretrieval):
            self.varname = name_str
            self.fname = 'readings_'+ name_str +'.csv'
            self.dbase_str = 'select * from ' + dbase_str
            self.fullretrieval = fulldbretrieval
    
    def readdata(self,gn_user,gn_pw,gn_host,gn_dbase):
        if not self.fullretrieval :
            try:
                print("read the csv file ",self.fname)
                dm=pd.read_csv(self.fname,
                                   parse_dates=['datetime'],
                                   index_col = ['datetime'])
                a=max(dm.index)
                sql_select_Query=self.dbase_str + " WHERE time >'" + str(a) + "'"
                print(self.fname,"_",sql_select_Query)
            except:
                print("File not found, full database download starts now")
                sql_select_Query = self.dbase_str
                self.fullretrieval = True
        else:
            # Retrieve the rest from mysql database:
            print("retrieve database")
            sql_select_Query = self.dbase_str
            
        # retrieve data from MySQL database
        cnx = mysql.connector.connect(user=gn_user, password=gn_pw,
                                  host=gn_host,database=gn_dbase)
        cursor = cnx.cursor()
        cursor.execute(sql_select_Query)
        mysql_df = pd.DataFrame(cursor.fetchall())
        cnx.close()
        # format sql data frame
        #
        # Error if no update values available, TO DO
        mysql_df.columns = ['datetime', "counter"]
        mysql_df['datetime'] = pd.to_datetime(mysql_df['datetime'])
        mysql_df['variable'] = self.varname
        mysql_df = mysql_df.set_index('datetime')
    
        # concatenate readings from file & readings from database
        if not self.fullretrieval:
            rawdata_export = pd.concat([dm,mysql_df],sort=True)
        else:
            rawdata_export = mysql_df    
        # export to file
        dowrite = rawdata_export.to_csv(self.fname)
        print("Updated values exported to file :",self.fname)
        # difference for each timestamp wrt previous one
        rawdata_export['delta']=rawdata_export['counter'].diff()
    
        return(rawdata_export)

In [10]:
def removeIncomplete(df):
    df['datum'] = df.index.date
    lastday = df['datum'][df.index[-1]]
    print("laatste datapunt in deze set is : ",lastday)
    df_day = df[['delta']].resample('d').count().sort_values('delta')
    print("Nr dagen raw Temperature readings : ", len(df_day))

    treshold = 0.9 * 24 * 60 
    
    df_day = df_day[df_day['delta']>=treshold]
    daystokeep = df_day.index.date 
    # keep to day, but do not add it when it is already there
    daystokeep = set(np.append(daystokeep,lastday))

    print("Nr days too less datapoints : ", len(set(df['datum']))-len(daystokeep))
    print("Nr days remaining : ", len(daystokeep))

    print("Nr of datapoints before removal of days : ", len(df))
    excluded = len(df[~df['datum'].isin(daystokeep)])
    print("Nr of datapoints before removed  : ", excluded)
    df = df[df['datum'].isin(daystokeep)]
    print("Nr of datapoints remaining : ",len(df), " Equal to : ",round(len(df)/24/60,0), "days")
    return(df)


In [11]:
def clean_up_gas(df):
    # Gas readings blijken niet altijd even goed te zijn, af en toe zelfs een factor 10 te hoog. 
    # neem de mediaan van de laaste 20 absolute  gegevens en gebruik dat om hogere waarden weg te laten
    df.loc[df['counter']>df[-20:]['counter'].median(),'counter']=np.nan
    df['counter'] = df['counter'].fillna(method='ffill')
    df['delta']=df['counter'].diff()
    # huge stepchanges (delta) in gas
    hugestepdates = df.loc[df['delta'].abs()>1].index.date
    # verwijder de datapunten van de dagen met de grootste stap
    # df.loc[~df.g.isin(hugestepdates)]
    print("nr dagen met grote stappen in verbruik : ",len(set(hugestepdates)))
    df['datum'] = df.index.date
    print("nr dagen ruwe data : ",len(set(df['datum'])))
    df = df[~df['datum'].isin(hugestepdates)]
    print("nr dagen na verwijdering : ",len(set(df['datum'])))
    # Remove the individual datapoints with delta > treshold 
    # Remaining items with huge step in gasconsumption, that still can exists as earlier data points might not be available.
    #df[df['delta'].abs()>1]
    return(df)

In [6]:
if __name__ == "__main__":
    gn_user,gn_pw,gn_host,gn_dbase,gn_drpbx = readsettings()
    gas = MinuteData('gas','item0054',False)
    
    df_g = MinuteData.readdata(gas,gn_user,gn_pw,gn_host,gn_dbase)

FileNotFoundError: [Errno 2] No such file or directory: '/home/pi/settings_GN/GN_values.txt'

In [29]:
# huh = readsettings()
# a = '2020-12-28 00:00:00'
# sql_select_Query = "select * from item0054 " + "WHERE time > '" + str(a) + "'"
# cnx = mysql.connector.connect(user=huh['user'], password=huh['pw'],
#                               host=huh['host'],database=huh['dbase'])
# cursor = cnx.cursor()
# cursor.execute(sql_select_Query)
# mysql_df = pd.DataFrame(cursor.fetchall())
# cnx.close()
# # format sql data frame
# #
# # Error if no update values available, TO DO
# mysql_df.columns = ['datetime', "counter"]
# mysql_df['datetime'] = pd.to_datetime(mysql_df['datetime'])
# # mysql_df['variable'] = self.varname
# mysql_df = mysql_df.set_index('datetime')

In [33]:
# mysql_df.describe()

,counter
count,2579.000000
mean,3310.747423
std,5.344430
min,3302.892000
25%,3306.123000
50%,3312.281000
75%,3313.952000
max,3320.182000


In [6]:
# gas = MinuteData('gas','item0054',False)

In [7]:
# gas.fname

'readings_gas.csv'